# Exercise: Data Augmentation

## Specify and Compile the Model

In [1]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/ResNet-50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Make a new blank sequential model
my_new_model = Sequential()
# Add the ResNet model as the first layer, with all it's pre-trained weights,
# but cut off the top of it, so that I get all it's learning but none of it's
# answers - I'm going to use it's shape and component detection to learn to 
# identify different items - items I specify. 
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
# Add a final layer to the model, on top of the ResNet layer, which can learn to 
# add up the results of the ResNet layer and determine which of the two classes
# I define the input is. 
my_new_model.add(Dense(num_classes, activation='softmax'))

# layer 0 is the ResNet layer, and it's already changed - I don't want to 
# change it (it's already pretty good) and I probably don't have the computational 
# power to create it anyway. 
my_new_model.layers[0].trainable = False

# Now it's all set up, compile the model into something that can be trained and used. 
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Fit the Model Using Data Augmentation

In [2]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224

# Specify the values for all arguments to data_generator_with_aug.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                            horizontal_flip=True, 
                                            width_shift_range=0.2,
                                            height_shift_range=0.2)
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Specify which type of ImageDataGenerator above is to load in training data.
train_generator = data_generator_with_aug.flow_from_directory(
    directory='../input/dogs-gone-sideways/images/train',
    target_size=(image_size, image_size),
    batch_size=12,
    class_mode='categorical')

# Specify which type of ImageDataGenerator above is to load in validation data.
validation_generator = data_generator_no_aug.flow_from_directory(
    directory='../input/dogs-gone-sideways/images/val',
    target_size=(image_size, image_size),
    class_mode='categorical')

my_new_model.fit_generator(
    train_generator,
    epochs = 3, 
    steps_per_epoch=19,
    validation_data=validation_generator)

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.
Epoch 1/3
19/19 [==============================] - 120s 6s/step - loss: 0.4937 - acc: 0.7676 - val_loss: 0.3080 - val_acc: 0.8618
Epoch 2/3
19/19 [==============================] - 85s 4s/step - loss: 0.2577 - acc: 0.9121 - val_loss: 0.1776 - val_acc: 0.9447
Epoch 3/3
19/19 [==============================] - 87s 5s/step - loss: 0.1691 - acc: 0.9605 - val_loss: 0.1383 - val_acc: 0.9493
